In [ ]:
# install all i need
# python -m pip install requests

In [15]:
import requests
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
'''
Q1. Running Elastic
What's the version.build_hash value?
'''

'\nQ1. Running Elastic\n'

In [2]:
! curl http://127.0.0.1:9200

{
  "name" : "4a7d2faec117",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "BukzAOOjTrezqaJlv5cuWQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [14]:
# 2da opcion
from elasticsearch import Elasticsearch

elsr_client= Elasticsearch('http://127.0.0.1:9200')
info_cliente= es_client.info()
info_cliente

ObjectApiResponse({'name': '4a7d2faec117', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BukzAOOjTrezqaJlv5cuWQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
info_cliente['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

In [10]:
# Now let's get the FAQ data 

import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
'''
Q2. Indexing the data
Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:
'''

In [19]:
# Configuracion del index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions-hmk'
elsr_client.indices.create(index= index_name, body= index_settings)

# indexamos cada uno de los documentos 
for i in tqdm(documents):
    elsr_client.index(index=index_name, document=i)

# we use index function to adding the data to elastic


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:03<00:00, 262.80it/s]


In [ ]:
'''
Q3. Searching
Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".
What's the score for the top ranking result?
'''

In [29]:
query= 'How do execute a command on a Kubernetes pod?'
# delete the filter 

search_query= {
    "size": 5,
    
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

# busqueda
respuesta_search= elsr_client.search(index= index_name, body= search_query)

    
    # resultados_elasticsearch =[]

    # for i in respuesta_search['hits']['hits']:
    #     resultados_elasticsearch.append(i['_source'])
    
    # return resultados_elasticsearch

/tmp/ipykernel_89487/1610308297.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  respuesta_search= elsr_client.search(index= index_name, body= search_query)


In [32]:
respuesta_search['hits']['hits']

score= []
for element in respuesta_search['hits']['hits']:
    score.append(element['_score'])

score

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

In [33]:
 # What's the score for the top ranking result? 44.50556


In [ ]:
'''
Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?
'''

In [34]:
query= 'How do copy a file to a Docker container?'
course= "machine-learning-zoomcamp"

search_query= {
    "size": 5,
    
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }
}

# busqueda-search
result_search= elsr_client.search(index= index_name, body= search_query)

/tmp/ipykernel_89487/1056700717.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result_search= elsr_client.search(index= index_name, body= search_query)


In [42]:
resultados_ES= []
for i in result_search['hits']['hits']:
    resultados_ES.append(i['_source'])

questions =[]
for i in resultados_ES:
    questions.append(i['question'])

# the third questions is
print(questions[2])

How do I copy files from a different folder into docker container’s working directory?


In [ ]:
'''
Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)
'''

In [60]:
# results from question 4
results= resultados_ES[0:3]

# creamos el contexto de los resultados de la busqueda por elastic search
contexto= ""
for i in results:
    contexto = contexto + f"Q: {i['question']} \nA: {i['text']} \n\n" 

# resultados[0]['text']
print(contexto)

Q: How do I debug a docker container? 
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD) 

Q: How do I copy files from my local machine to docker container? 
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani 

Q: How do I copy files from a different folder into docker container’s working directory? 
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's ho

In [80]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context} 

""".strip() 
# strip: elimina espacios en blanco, al inicio y al final
prompt_final= prompt_template.format(question= query, context= contexto).strip() 

In [81]:
print(prompt_final)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT: 
Q: How do I debug a docker container? 
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD) 

Q: How do I copy files from my local machine to docker container? 
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/con

In [82]:
len(prompt_final)

1452

In [ ]:
'''
Q6.Tokens
'''

In [74]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 28.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [75]:
import tiktoken

In [76]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [77]:
len(encoding.encode(prompt_final))

325

In [79]:
print(prompt_final)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.


QUESTION: How do copy a file to a Docker container?

CONTEXT: 
Q: How do I debug a docker container? 
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD) 

Q: How do I copy files from my local machine to docker container? 
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/co